In [1]:
from dune_client.client import DuneClient
dune = DuneClient("rVRuwpjjYaD1yXJAVH3dVZufZcESZK4E")
query_result = dune.get_latest_result(4816328)



In [6]:
import pandas as pd
# 提取 rows 数据
rows = query_result.result.rows

# 创建 DataFrame
df = pd.DataFrame(rows)

# 如果需要，将 transaction_date 列转换为 datetime 类型
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# 查看 DataFrame
df

,cumulative_unique_addresses,transaction_date,unique_addresses
0,257447,2024-08-29,257447
1,344109,2024-08-30,135272
2,412628,2024-08-31,140328
3,502964,2024-09-01,180400
4,544539,2024-09-02,126008
...,...,...,...
185,18521286,2025-03-02,542165
186,18781478,2025-03-03,532782
187,18954839,2025-03-04,439754
188,19059312,2025-03-05,371240


In [7]:
df.to_csv('kaia.csv', index=False)
